<a href="https://colab.research.google.com/github/Neeraj-007/TAB-015/blob/main/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Importing Modules**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [3]:
%matplotlib inline

## **Data Gathering**

In [6]:
path = r'/content/drive/MyDrive/AutoBot Round 1/dataset'

In [8]:
l=os.listdir(path)

In [9]:
data=[]

for img in l:
    img_path=os.path.join(path,img)
    arr=cv2.imread(img_path,1)
    if arr is not None:
      arr=cv2.cvtColor(arr,cv2.COLOR_BGR2RGB)
      arr_new=cv2.resize(arr,(100,100))
      img1=img.split(' ')
      img1=img1[0]
      if(img1=='animal'):
          label=0
      elif(img1 == 'person'):
          label=1
      elif(img1 == 'cones'):
          label=2
      elif(img1 == 'stop'):
          label=3
      elif(img1 == 'traffic_red'):
          label=4
      elif(img1 == 'traffic_blue'):
          label=5
      else:
          label=6
      data.append([arr_new, label])

## **Shuffling the continuous arrangement data**

In [11]:
import sklearn
import random
import shutil
import sklearn.model_selection
x=[]
y=[]

for features, label in data:
    x.append(features)
    y.append(label)

x=np.array(x)
y=np.array(y)


## **Dividing the dataset into training and testing**

In [12]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 42)

In [13]:
X_train.shape

(5480, 100, 100, 3)

In [14]:
X_test.shape

(1370, 100, 100, 3)

In [15]:
X_train = X_train/255

## **Train Model**

In [16]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2)))

model.add(tf.keras.layers.Conv2D(64, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), padding = 'same'))

model.add(tf.keras.layers.Conv2D(32, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), padding = 'same'))

model.add(tf.keras.layers.Conv2D(64, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), padding = 'same'))

model.add(tf.keras.layers.Conv2D(32, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), padding = 'same'))

model.add(tf.keras.layers.Conv2D(64, 2, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), padding = 'same'))

model.add(tf.keras.layers.Dense(1024, activation ='relu'))
model.add(tf.keras.layers.Dropout(0.8))

model.add(tf.keras.layers.Dense(7, activation ='softmax'))



In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy())

In [18]:
X_train.shape

(5480, 100, 100, 3)

In [19]:

np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6])

In [20]:
model.fit(x = X_train, y = y_train, epochs = 10, batch_size = 64)

Epoch 1/10
86/86 [==============================] - 53s 600ms/step - loss: 1.7609
Epoch 2/10
86/86 [==============================] - 51s 596ms/step - loss: 1.7195
Epoch 3/10
86/86 [==============================] - 51s 589ms/step - loss: 1.7151
Epoch 4/10
86/86 [==============================] - 51s 593ms/step - loss: 1.7152
Epoch 5/10
86/86 [==============================] - 50s 584ms/step - loss: 1.7084
Epoch 6/10
86/86 [==============================] - 51s 589ms/step - loss: 1.7076
Epoch 7/10
86/86 [==============================] - 51s 588ms/step - loss: 1.7022
Epoch 8/10
86/86 [==============================] - 50s 583ms/step - loss: 1.7014
Epoch 9/10
86/86 [==============================] - 50s 584ms/step - loss: 1.6998
Epoch 10/10
86/86 [==============================] - 50s 583ms/step - loss: 1.6992


In [21]:
!mkdir -p saved_model
model.save('saved_model/model.h5')

In [25]:

new_model = tf.keras.models.load_model('saved_model/model.h5')
new_model.summary()
# Check its architecture


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 99, 99, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 32)        8224      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0

In [26]:
model.evaluate(x = X_test, y = y_test, batch_size = 64)

22/22 [==============================] - 4s 163ms/step - loss: 1.6979


1.6979451179504395

In [27]:
predictions = model.predict(X_test[:10])
predictions[0]

array([[[0.22301047, 0.29719368, 0.23623192, 0.09419093, 0.05171363,
         0.05053916, 0.04712011]]], dtype=float32)